# Creating features

In [12]:
import cv2
import numpy as np
import os
import radialProfile
import glob
from matplotlib import pyplot as plt
import pickle

path = ['thispersondoesntexists_10K/', '100KFake_10K/','Flickr-Faces-HQ_10K/', 'celebA-HQ/']
labels = [1,1,0,0]
epsilon = 1e-8
data = {}
#number of samples from each dataset
stop = 250
number_iter = 4 * stop
psd1D_total = np.zeros([number_iter, 722])
label_total = np.zeros([number_iter])
iter_ = 0

for z in range(4):
    
    psd1D_average_org = np.zeros(722)
    
    for filename in glob.glob(path[z]+"/*.jpg"):
        
        img = cv2.imread(filename,0)
        # Calculate FFT
        f = np.fft.fft2(img)
        fshift = np.fft.fftshift(f)
        fshift += epsilon
        magnitude_spectrum = 20*np.log(np.abs(fshift))
        # Calculate the azimuthally averaged 1D power spectrum
        psd1D = radialProfile.azimuthalAverage(magnitude_spectrum)

        psd1D_total[iter_,:] = psd1D
        label_total[iter_] = labels[z]

        iter_+=1
        if iter_ >= stop:
            break

data["data"] = psd1D_total
data["label"] = label_total

output = open('dataset_freq_1000.pkl', 'wb')
pickle.dump(data, output)
output.close()
print("DATA Saved")    

DATA Saved


# Classification

In [11]:
import pandas as pd
import numpy as np
import pickle

#number of runs
num = 5
SVM = 0
LR = 0

for z in range(num):
    # read python dict back from the file
    pkl_file = open('dataset_freq_1000.pkl', 'rb')
    
    data = pickle.load(pkl_file)
    pkl_file.close()
    X = data["data"]
    y = data["label"]
    X.shape
    try:

        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

        from sklearn.svm import SVC
        svclassifier = SVC(kernel='linear')
        svclassifier.fit(X_train, y_train)
        #print('Accuracy on test set: {:.3f}'.format(svclassifier.score(X_test, y_test)))
               
        
        from sklearn.linear_model import LogisticRegression
        logreg = LogisticRegression(solver='liblinear', max_iter=1000)
        logreg.fit(X_train, y_train)
        #print('Accuracy on test set: {:.3f}'.format(logreg.score(X_test, y_test)))
        
        SVM+=svclassifier.score(X_train, y_train)
        LR+=logreg.score(X_test, y_test)
        
    except:
        num-=1
        print(num)
    

print("Average SVM: "+str(SVM/num))
print("Average LR: "+str(LR/num))

Average SVM: 1.0
Average LR: 1.0
